# ACDtools dev sandbox 

#### Date: 2 October, 2024

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

# Install ACDtools

In [1]:
!pip install --user -e /g/data/es60/users/thomas_moore/code/ACDtools

Obtaining file:///g/data/es60/users/thomas_moore/code/ACDtools
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ACDtools (pyproject.toml) ... done
  Created wheel for ACDtools: filename=ACDtools-0.1-0.editable-py3-none-any.whl size=3514 sha256=860f9930eb704c3c8d1d81f189a712a456deb39dba756e2bceb590f2b5c63b32
  Stored in directory: /jobfs/126318436.gadi-pbs/pip-ephem-wheel-cache-1ymnjtvy/wheels/b6/a3/f2/6ce45fbdc116ad50e421d6a11cb060cc796e867501807af446
Successfully built ACDtools
  Attempting uninstall: ACDtools
    Found existing installation: ACDtools 0.1
    Uninstalling ACDtools-0.1:
      Successfully uninstalled ACDtools-0.1


In [72]:
# Enable autoreload in the notebook
%load_ext autoreload
%autoreload 1 
%aimport ACDtools.util
# Importing from your local package
from ACDtools.util import test_function
from ACDtools.util import detect_compute_platform
from ACDtools.util import load_config
from ACDtools.util import start_dask_cluster_from_config
from ACDtools.util import esm_datastore_unique

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Notebook settings

### filter warnings

In [3]:
import warnings
warnings.filterwarnings("ignore") # Suppress warnings

# Dask cluster from config
`client, cluster = start_dask_cluster_from_config('netcdf_work')`
<br>OR<br>
`client, cluster = start_dask_cluster_from_config('zarr_work')`

In [4]:
client, cluster = start_dask_cluster_from_config('netcdf_work')

Cluster started with 28 workers.
Dashboard available at: /proxy/8787/status


# Issue: write function to load ACCESS-ESM1.5 data object using intake catalogs at NCI
- https://github.com/Thomas-Moore-Creative/ACDtools/issues/1

## utilise CMIP6 data catalogs for NCI holdings

##### Information on climate data catalogs across Australian HPC

**ACCESS-NRI** https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/how.html <br>
**NCI** https://opus.nci.org.au/pages/viewpage.action?pageId=213713098


##### $\bigstar$ Get inspiration from ACCESS-NRI intake catalog docs: ACCESS-ESM1-5 CMIP6 example
https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/quickstart.html

# import packages

In [40]:
import intake
import xarray as xr
import numpy as np
import gc
import json

### import the ACCESS-NRI catalog

In [6]:
catalog = intake.cat.access_nri

### (1) "I know I want Australian CMIP6 data - so that's fs38 and I need access to that NCI project"

In [7]:
cmip6_fs38_datastore = catalog.search(name='cmip6_fs38').to_source()

### (2) "what are the realms covered by cmip6_fs38?"

In [80]:
esm_datastore_unique(cmip6_fs38_datastore,keep_list=['realm'])

╒════════════╤═════════════════╕
│ Category   │ Unique values   │
╞════════════╪═════════════════╡
│ realm      │ aerosol         │
│            │ atmos           │
│            │ land            │
│            │ landIce         │
│            │ ocean           │
│            │ ocnBgchem       │
│            │ seaIce          │
╘════════════╧═════════════════╛


### (3) I want to see what variables, over what frequencies, are available in both the 'ocean' & 'oceanBgchem' realms

In [81]:
cmip6_fs38_ocean_datastore = cmip6_fs38_datastore.search(realm=['ocean','ocnBgchem'])

In [82]:
[sorted_unique_dict, table_data] = esm_datastore_unique(cmip6_fs38_ocean_datastore,return_results=True)

╒════════════════╤═══════════════════════╕
│ Category       │ Unique values         │
╞════════════════╪═══════════════════════╡
│ experiment_id  │ 1pctCO2               │
│                │ 1pctCO2-bgc           │
│                │ 1pctCO2-cdr           │
│                │ 1pctCO2-rad           │
│                │ abrupt-4xCO2          │
│                │ esm-1pct-brch-1000PgC │
│                │ esm-1pct-brch-2000PgC │
│                │ esm-1pct-brch-750PgC  │
│                │ esm-hist              │
│                │ esm-pi-CO2pulse       │
│                │ esm-pi-cdr-pulse      │
│                │ esm-piControl         │
│                │ esm-ssp585            │
│                │ esm-ssp585-ssp126Lu   │
│                │ faf-all               │
│                │ faf-heat              │
│                │ faf-heat-NA0pct       │
│                │ faf-heat-NA50pct      │
│                │ faf-passiveheat       │
│                │ faf-stress            │
│          

In [83]:
search = cmip6_fs38_ocean_datastore.search(file_type='l',
                    variable_id='intpp',source_id='ACCESS-ESM1-5',experiment_id='historical')

In [84]:
esm_datastore_unique(search)

╒════════════════╤═════════════════╕
│ Category       │ Unique values   │
╞════════════════╪═════════════════╡
│ experiment_id  │ historical      │
├────────────────┼─────────────────┤
│ file_type      │ l               │
├────────────────┼─────────────────┤
│ frequency      │ mon             │
├────────────────┼─────────────────┤
│ grid_label     │ gn              │
├────────────────┼─────────────────┤
│ institution_id │ CSIRO           │
├────────────────┼─────────────────┤
│ project_id     │ CMIP            │
├────────────────┼─────────────────┤
│ realm          │ ocnBgchem       │
├────────────────┼─────────────────┤
│ source_id      │ ACCESS-ESM1-5   │
├────────────────┼─────────────────┤
│ table_id       │ Omon            │
├────────────────┼─────────────────┤
│ variable_id    │ intpp           │
╘════════════════╧═════════════════╛


In [86]:
esm_datastore_unique(search,keep_list=['path'])

╒════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ Category   │ Unique values                                                                                                                                                           │
╞════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ path       │ /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r10i1p1f1/Omon/intpp/gn/v20200605/intpp_Omon_ACCESS-ESM1-5_historical_r10i1p1f1_gn_185001-201412.nc │
│            │ /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r11i1p1f1/Omon/intpp/gn/v20200803/intpp_Omon_ACCESS-ESM1-5_historical_r11i1p1f1_gn_185001-201412.nc │
│            │ /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/his

In [ ]:
cmip6_datastore_filtered = cmip6_datastore.search(
    source_id="ACCESS-ESM1-5", 
    table_id="Omon", 
    variable_id="tos", 
    experiment_id="historical", 
    file_type="l"
)

cmip6_datastore_filtered